# Scripts and Drafts for Arrow-Debreu Market

In [44]:
using Pkg
Pkg.activate("../")
using Revise
using Random, SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf
import MathOptInterface as MOI

using ExchangeMarket
using DRSOM

include("../tools.jl")
include("../plots.jl")
include("./func_regnew.jl")
switch_to_pdf(; bool_use_html=true)

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


:html

## Test run

In [45]:
Random.seed!(1)
m = 10
n = 10
ρ = -99
# b = rand(n, m)
b = diagm(ones(n))
# b ./= sum(b; dims=2)
e = ones(n)

# -----------------------------------------------------------------------
# setup market
# -----------------------------------------------------------------------
f0 = ArrowDebreuMarket(m, n; b=b, ρ=ρ, scale=30.0, sparsity=0.99)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct

ArrowDebreuMarket initialization started...
ArrowDebreuMarket initialized in 8.392333984375e-5 seconds
ArrowDebreuMarket initialization started...
ArrowDebreuMarket initialized in 8.606910705566406e-5 seconds


:direct

In [46]:
traj_p, traj_beta, traj_residual, traj_c, traj_kappa = regnew3(
    f1, alg;
    K=100, log_interval=10,
    bool_proj=false,
    scaler=1,
    τ=0.05
)
fig = generate_empty(; shape=:wide)
plot!(
    fig,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=(-36:1:0) .|> exp,
    xtickfont=font(18),
    ytickfont=font(18),
    yscale=:log,
    size=(700, 600),
)
plot!(fig, (traj_beta.+1e-20)[1:1:end], label=L"n: %$(n)", linewidth=2, linestyle=:dash, markershape=:circle, markerstrokecolor=nothing, markerstrokewidth=0.0)
savefig(fig, "/tmp/arrow_beta_$(n).pdf")

alg.p = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
10th iteration |p|₁: 1.0e+00 sum(q): 2.6e-19
   |-10th               p: [9.5e-02, 1.0e-01]
   |-10th               r: 1.8e-02  λ₁: [0.0e+00, 1.1e-02]
   |-10th               λ: 1.3e-02 |v|: 8.1e-03 ∈ [8.1e-03, 8.2e-03)]
   |-10th            |a|ₚ: 1.1e-04 |z|ₚ: 1.3335e-02
   |-10th       |a|ₚ/|z|ₚ: 8.3e-03 κ: 1.3e+00
   |-10th              αₘ: 1.0e+00
20th iteration |p|₁: 1.0e+00 sum(q): -5.3e-20
   |-20th               p: [9.0e-02, 1.0e-01]
   |-20th               r: 1.6e-02  λ₁: [0.0e+00, 1.0e-02]
   |-20th               λ: 1.3e-02 |v|: 7.8e-03 ∈ [7.8e-03, 7.9e-03)]
   |-20th            |a|ₚ: 1.0e-04 |z|ₚ: 1.2685e-02
   |-20th       |a|ₚ/|z|ₚ: 8.0e-03 κ: 1.3e+00
   |-20th              αₘ: 1.0e+00
30th iteration |p|₁: 1.0e+00 sum(q): 2.1e-19
   |-30th               p: [8.5e-02, 1.1e-01]
   |-30th               r: 1.5e-02  λ₁: [0.0e+00, 9.9e-03]
   |-30th               λ: 1.2e-02 |v|: 7.6e-03 ∈ [7.6e-03, 7.6e-03)]
   |-30th     

"/tmp/arrow_beta_10.pdf"

In [60]:
alg.p .= 100
update_budget!(f1, alg.p)
play!(alg, f1)
grad!(alg, f1)
eval!(alg, f1)
hess!(alg, f1)
_J = alg.H
_z = alg.∇
_r = sum(_z .^ 2)

_P = diagm(alg.p)
_PJ = _P * _J
_pz = alg.p .* _z
_a = _PJ' * _pz
_pa = alg.p .* _a
_na = norm(_pa)

118.83076211458437

In [32]:
alg.∇

1000-element Vector{Float64}:
 -1.1102230246251565e-15
 -2.220446049250313e-16
  3.3306690738754696e-16
 -8.881784197001252e-16
  5.551115123125783e-16
  0.0
  1.1102230246251565e-16
  9.992007221626409e-16
 -2.220446049250313e-16
  2.220446049250313e-16
  ⋮
  4.440892098500626e-16
  4.440892098500626e-16
  2.220446049250313e-16
  2.220446049250313e-16
 -2.220446049250313e-16
  8.881784197001252e-16
  5.551115123125783e-16
 -4.440892098500626e-16
  0.0

In [62]:
sum(alg.p)

1000.0

In [63]:
# afscaling(H) = diagm(alg.p) * H * diagm(alg.p)

Jp = afscaling(alg.H)
Pip = I(n) - alg.p * alg.p' / (norm(alg.p)^2)

10×10 Matrix{Float64}:
  0.9  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1
 -0.1   0.9  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1   0.9  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1   0.9  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1   0.9  -0.1  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1  -0.1   0.9  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1  -0.1  -0.1   0.9  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1   0.9  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1   0.9  -0.1
 -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1   0.9

In [66]:
Jp

10×10 Matrix{Float64}:
 -0.0408467  -0.842835  -0.817157  …   9.12181   -0.759671  -2.09522
 -0.786148    1.44828    0.271823     -0.777583   0.450489  -0.787944
 -0.910993    0.305848   1.48814      -0.700201   0.370366  -0.753795
 -0.928389    0.30687    0.235073     -0.816095   0.38207   -0.829198
 -0.942474    0.295285   0.320236     -0.660911   0.37004   -0.797708
 -0.796559    0.379218   0.682236  …  -0.733865   0.222729  -0.907667
 -0.79738     0.253924   0.232943     -0.449001   0.385294  -0.823494
 -0.938605    0.378775   0.280342      0.204917   0.498967  -0.783021
 -0.936941    0.422731   0.454626     -0.797475   1.43753   -0.794308
 -2.11849    -0.754727  -0.955772     -1.94841   -0.837699   0.169484

In [65]:
eigen(Jp * Pip*Pip*Jp')

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
10-element Vector{Float64}:
   2.247158679986262e-15
   0.04826423683708257
   0.679540265301505
   0.8457974086890816
   1.2103993574717131
   1.4067554270579175
   1.7455034428179284
  19.368290768295125
  74.26230635932603
 130.11320298825956
vectors:
10×10 Matrix{Float64}:
  0.0471288  -0.071607     0.00416794  …  -0.285446   0.658259    0.690619
  0.133882   -0.0624032   -0.307499       -0.30645   -0.172577    0.00293103
  0.0191992  -0.0395123   -0.490039       -0.323173  -0.165209    0.00467618
 -0.674986   -0.603114     0.0568481      -0.328702  -0.0839532  -0.0654676
  0.37341     0.0708648    0.202704       -0.337596  -0.163663    0.00673376
  0.126637   -0.00323855   0.468389    …  -0.315407  -0.175533    0.0121233
  0.222337    0.0194856   -0.469274       -0.308756  -0.133226    0.0173042
 -0.519408    0.776847     0.056134       -0.321653  -0.0731022   0.0545304
  0.207692   -0.120869     0.422114       -0.3

## Plots by $\rho$

In [ ]:
results = []
n = m = 50
ρs = [-50, -40, -30, -20, -10, -1, 0.96]
for ρ in ρs
    b = diagm(ones(n))
    e = ones(n)

    # -----------------------------------------------------------------------
    # setup market
    # -----------------------------------------------------------------------
    f0 = ArrowDebreuMarket(m, n; b=b, ρ=ρ, scale=30.0, sparsity=0.99)
    linconstr = LinearConstr(1, n, ones(1, n), [1.0])

    # -----------------------------------------------------------------------
    # compute ground truth
    # -----------------------------------------------------------------------
    f1 = copy(f0)
    p₀ = ones(n) ./ (n)
    x₀ = ones(n, m) ./ m
    f1.x .= x₀
    alg = HessianBar(
        n, m, p₀;
        linconstr=linconstr,
    )
    alg.linsys = :direct
    traj_p, traj_beta, traj_residual, traj_c, traj_kappa = regnew3(
        f1, alg; log_interval=50, K=5000,
        scaler=10,
        τ=1.01,
    )
    push!(results, [traj_p, traj_beta, traj_residual, traj_c, traj_kappa])
end
colors = [
    "#000000",  # black
    "#08306b",  # very dark blue
    "#08519c",  # dark blue
    "#2171b5",  # medium-dark blue
    "#4292c6",  # medium blue
    "#6baed6",  # medium-light blue
    "#9ecae1",  # light blue
    "#c6dbef",  # soft blue
    "#dbe9f6"   # pale but still visible
]
fig_beta = generate_empty(; shape=:wide)
plot!(
    fig_beta,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)
for (idx, (traj_p, traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _ρ = ρs[idx]
    plot!(fig_beta, traj_beta .+ 1e-20, label=L"\rho: %$(_ρ)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
end
savefig(fig_beta, "/tmp/arrow_beta.pdf")


### Plot c and kappa

In [ ]:
ffs = []

fig_kappa = plot(
    labelfontsize=20,
    xtickfont=font(16),
    ytickfont=font(16),
    legendfontsize=20,
    titlefontsize=20,
    legend=:topright,
    legendfonthalign=:left,
    linewidth=2,
)
plot!(
    fig_kappa,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)

for (idx, (traj_p, traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _ρ = ρs[idx]
    _l = length(traj_kappa)
    plot!(fig_kappa, traj_kappa[max(1, _l - 50):end], label=L"\rho: %$(_ρ)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
end
push!(ffs, fig_kappa)


In [ ]:
savefig(fig_kappa, "/tmp/arrow_kappa.pdf")

### Plot c

In [ ]:
fig_c = generate_empty(; shape=:wide)
plot!(
    fig_c,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)

for (idx, (traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _ρ = ρs[idx]
    plot!(fig_c, traj_c[1:end-1], label=L"\rho: %$(_ρ)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
end

In [ ]:
savefig(fig_c, "/tmp/arrow_c.pdf")

## Plots by $n$

In [ ]:
results = []
# m = 1000
ns = [80, 40, 20, 10]
ρ = -10
Random.seed!(1)
for n in ns
    # Random.seed!(1)
    m = n
    # b = rand(n, m)
    # b ./= sum(b; dims=2)
    b = diagm(ones(n))
    e = ones(n)
    Π = I - e * e' / n

    # -----------------------------------------------------------------------
    # setup market
    # -----------------------------------------------------------------------
    f0 = ArrowDebreuMarket(m, n; b=b, ρ=ρ, scale=30.0, sparsity=0.9)
    linconstr = LinearConstr(1, n, ones(1, n), [1.0])

    # -----------------------------------------------------------------------
    # compute ground truth
    # -----------------------------------------------------------------------
    f1 = copy(f0)
    p₀ = ones(n) ./ (n)
    x₀ = ones(n, m) ./ m
    f1.x .= x₀
    alg = HessianBar(
        n, m, p₀;
        linconstr=linconstr,
    )
    alg.linsys = :direct
    traj_p, traj_beta, traj_residual, traj_c, traj_kappa = regnew3(f1, alg; K=5000, log_interval=100, bool_proj=true)
    push!(results, [traj_p, traj_beta, traj_residual, traj_c, traj_kappa])
end


### Plot beta

In [ ]:
colors = [
    "#000000",  # black
    "#08306b",  # very dark blue
    "#08519c",  # dark blue
    "#2171b5",  # medium-dark blue
    "#4292c6",  # medium blue
    "#6baed6",  # medium-light blue
    "#9ecae1",  # light blue
    "#c6dbef",  # soft blue
    "#dbe9f6"   # pale but still visible
]
fig_beta = generate_empty(; shape=:wide)
plot!(
    fig_beta,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)
for (idx, (traj_p, traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _n = ns[idx]
    _l = length(traj_beta)
    _y = (traj_beta .+ 1e-20) .* _n
    println("$(_n)th iteration: $(traj_beta[end])")
    if _l > 1000
        plot!(fig_beta, 1:20:_l, _y[1:20:end], label=L"n: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
    else
        plot!(fig_beta, _y, label=L"n: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
    end
end
savefig(fig_beta, "/tmp/arrow_beta_n_$ρ.pdf")

### Plot c and kappa

In [ ]:
ffs = []

fig_kappa = plot(
    labelfontsize=20,
    xtickfont=font(16),
    ytickfont=font(16),
    legendfontsize=20,
    titlefontsize=20,
    legend=:topright,
    legendfonthalign=:left,
    linewidth=2,
)
plot!(
    fig_kappa,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)

for (idx, (traj_p, traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _n = ns[idx]
    plot!(fig_kappa, traj_kappa[1:end-1], label=L"n: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
    _l = length(traj_kappa)
    # plot!(fig_kappa, traj_kappa[max(1, _l - 100):end], label=L"\rho: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
end
push!(ffs, fig_kappa)
savefig(fig_kappa, "/tmp/arrow_kappa_n.pdf")


In [ ]:
### focus on n=50
fig_kappa = plot(
    labelfontsize=20,
    xtickfont=font(16),
    ytickfont=font(16),
    legendfontsize=20,
    titlefontsize=20,
    legend=:topright,
    legendfonthalign=:left,
    linewidth=2,
)
plot!(
    fig_kappa,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)
_n = ns[4]
plot!(fig_kappa, results[4][end][end-100:end-1], label=L"n: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, markerstrokecolor=nothing, markerstrokewidth=0.0)
savefig(fig_kappa, "/tmp/arrow_kappa_$(_n).pdf")


### Plot c

In [ ]:
fig_c = generate_empty(; shape=:wide)
plot!(
    fig_c,
    legend=:topright,                   # place legend on top
    legend_columns=4,                  # two columns
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    # yticks=10.0 .^ (-36:4:3),
    xtickfont=font(18),
    ytickfont=font(18),
    # xscale=:log10,
    size=(700, 600),
)

for (idx, (traj_p, traj_beta, traj_residual, traj_c, traj_kappa)) in enumerate(results)
    _n = ns[idx]
    plot!(fig_c, traj_c[1:end-1], label=L"n: %$(_n)", linewidth=2, linestyle=:dash, markershape=:circle, color=colors[idx], markerstrokecolor=nothing, markerstrokewidth=0.0)
end
savefig(fig_c, "/tmp/arrow_c_n.pdf")

## 2D example

In [ ]:
Random.seed!(1)
m = 2
n = 2
ρ = -10.0
# b = rand(n, m)
b = diagm(ones(n))
# b ./= sum(b; dims=2)
e = ones(n)
ϵ = 1e-3


# c = [
#     1 ϵ
#     ϵ 2ϵ
# ] .* 100
# f0 = ArrowDebreuMarket(m, n; c=c, b=b, ρ=ρ, scale=30.0, sparsity=1.0)
f0 = ArrowDebreuMarket(m, n; b=b, ρ=ρ, scale=30.0, sparsity=1.0)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct
traj_p, traj_beta, traj_residual, traj_c, traj_kappa = regnew3(
    f1, alg; K=2000, log_interval=1, bool_proj=false
)


In [ ]:
using Plots
gr()

function extract_xy(traj)
    if traj isa AbstractMatrix
        if size(traj, 1) >= 2
            return collect(traj[1, :]), collect(traj[2, :])
        else
            return collect(traj[:, 1]), collect(traj[:, 2])
        end
    else
        x = [p[1] for p in traj]
        y = [p[2] for p in traj]
        return x, y
    end
end

x_all, y_all = extract_xy(traj_p)

x_min, x_max = extrema(x_all)
y_min, y_max = extrema(y_all)

anim = @animate for k in 1:length(x_all)
    plot(
        x_all[1:k],
        y_all[1:k],
        lw=2,
        label=false,
        xlim=(x_min, x_max),
        ylim=(y_min, y_max),
        aspect_ratio=:equal,
        marker=:circle,
        markersize=3,
    )
    scatter!(x_all[k:k], y_all[k:k], color=:red, label=nothing)
end

gif(anim, "traj_p.gif", fps=1)


In [ ]:
traj_p

# Some trials on the Jacobian

In [ ]:
alg.pb

In [ ]:
afscaling(H) = diagm(alg.pb) * H * diagm(alg.pb)

Jp = afscaling(alg.H)

In [ ]:
(Jp + 0.1I) \ ones(n)

### Test linear system

In [ ]:
n = 5
A = J1
S = diagm(rand(n) .+ 1.0)
P = diagm(rand(n))
feasd = randn(n)
feasc = randn(n)
feasp = 1 - sum(P)

In [ ]:
# via left singular vector for smallest singular value
U, S, V = svd(A)
w = U[:, end]                 # A' * w ≈ 0
w /= dot(w, e)                # normalize so w' e = 1

In [ ]:
w' * (1 .- f1.x[:, 1])